## Importing packages

In [23]:
import pandas as pd
from pandas.api.types import is_string_dtype
import numpy as np

## Calculate pearson correlation

In [24]:
def calc_pearson_correlation(df, goal):
    df = df.replace({'international plan': {'yes': True,
                                    'no': False}})
    df = df.replace({'voice mail plan': {'yes': True,
                                'no': False}})
    corr_arr = []
    for col in df.columns:
        if not is_string_dtype(df[col]):  # failing with conversion error on str types
            corr_arr.append(df[goal].corr(df[col], method ='pearson', min_periods=1))
    return corr_arr

df = pd.read_csv('churn_data.csv')
goal = 'churn'
res = calc_pearson_correlation(df, goal)

## group_customer_service_calls

In [38]:
def group_customer_service_calls(df, value_column):
    # calculate the percentage mappings and derive the conditions based on it (I so it manually, but probably there is a better, automatic way)
    calls_percentage_mappings = df[value_column].value_counts(normalize=True) * 100
    #create a list of our conditions
    conditions = [
        (df['customer service calls'] >= 3),
        (df['customer service calls'] == 1),
        (df['customer service calls'] == 2) | (df['customer service calls'] == 0)]

    # create a list of the values we want to assign for each condition
    values = ['High', 'Low', 'Medium']
    df['group'] = np.select(conditions, values)
    return df

value_column = 'customer service calls'
res_df = group_customer_service_calls(df, value_column)

## calc_probability

In [35]:
def calc_probability(df, column_a, column_b, goal):
    res_pivot = pd.pivot_table(
        df,
        values=goal,
        index=[column_a],
        columns=[column_b],
        aggfunc=lambda x: x.map(int).mean(),
        fill_value=0
    )
    return res_pivot

column_a = 'voice mail plan'
column_b = 'international plan'
goal = 'churn'
res_pivot = calc_probability(df, column_a, column_b, goal)